

```
                                     Assignment 4 Report
          
         Student Name:
         Yara Hazem Mohamed     7592
         Hana waleed            7599
         Sohail waleed          7372
   
     

     Introduction:
      Considering 3 by 3 Markov decision process we will use value iteration and policy iteration to evaluate optimal policy for different
      reward values.
```



In [1]:
REWARD = -1
GAMMA = 0.99


NUM_ACTIONS = 4
ACTIONS = [(1,0),(0,1),(-1,0),(0,-1)]
# down left up right
ACTIONS_TEXT = ["DOWN", "RIGHT", "UP", "LEFT"]

NUM_ROWS = 3
NUM_COLS = 3


## Part 1 : value iteration
iterative application of Bellman optimality backup
At each iteration k +1
- For all states s ∈ S Update vk+1(s) from vk(s)
- Unlike policy iteration, there is no explicit policy Intermediate value functions may not correspond to any policy
- will converge into optimal policy

In [2]:
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))

Math(r"v_{k+1}(s) = \max_{a \in A} \left( R_a^s + \gamma \sum_{s' \in S} P_{ss'}^a \cdot v_k(s') \right)")

<IPython.core.display.Math object>

In [3]:
#Calculate new value of state given next action
#Pass action / next action as numbers between 0 and 3
def get_v (env_v,row,col,next_action):
    """
    Calculate v*(s')
    """
    dr = ACTIONS[next_action][0]
    dc = ACTIONS[next_action][1]
    newR = row + dr
    newC = col + dc
    if newR < 0 or newC < 0 or newR >= NUM_ROWS or newC >= NUM_ROWS :
        return env_v[row][col]
    else:
        return env_v[newR][newC]



In [4]:
def calc_v (env_v ,row, col, action):
    """
    Rs a + GAMMA sum (Pss' a) v*(s')
    """
    v = 0
    v += 0.1 * get_v(env_v, row, col, (action-1)%NUM_ACTIONS)
    v += 0.8 * get_v(env_v, row, col, action)
    v += 0.1 *  get_v(env_v, row, col, (action+1)%NUM_ACTIONS)
    v *= GAMMA
    r = -1
    if (row == 0 and col == 0) or (row == 0 and col == 2):
        r = ENV_R[row][col]
    v+=r
    return v

In [5]:
def value_iteration(env_v,N):
    """
    Bellman Update equation
    """
    for _ in range (0,N):
        nextv = env_v

        for r in range(NUM_ROWS):
            for c in range(NUM_COLS):
                if (r == 0 and c == 0) or (r == 0 and c == 2):
                    continue
                nextv[r][c] = max([calc_v(env_v, r, c, action) for action in range(NUM_ACTIONS)])

        env_v = nextv

    return env_v


In [6]:
def get_policy(env_v):
    policy = [[-1, -1, -1] for i in range(NUM_ROWS)]
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS):
            if (r == 0 and c == 0) or (r == 0 and c == 2):
                continue
            # Choose the action that maximizes the utility
            maxAction, maxU = None, -float("inf")
            for action in range(NUM_ACTIONS):
                u = calc_v(env_v, r, c, action)
                if u > maxU:
                    maxAction, maxU = action, u
            policy[r][c] = maxAction
    policy_text = [["Terminal","Terminal", "Terminal"] for i in range(NUM_ROWS)]
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS):
            if policy[r][c] == -1:
                continue
            else:
                policy_text[r][c] = ACTIONS_TEXT [policy[r][c]]

    return policy_text



In [7]:
def print_2d_grid(grid, N):
    print("+" * 4 * N)
    for row in grid:
        print(" | ".join(["{:^{width}}".format(str(elem), width=N) for elem in row]))
    print("+" * 4 * N )

diff_r = [100,3,0,-3]

for r in diff_r:
    print(f"FOR R = {r}")
    ENV_R = [[r,-1,10], [-1,-1,-1], [-1,-1,-1]]
    env_v = [[r,0,10],[0,0,0],[0,0,0]]
    V = value_iteration(env_v, 100)
    print_2d_grid(V,20)
    print("\n")
    p = get_policy(V)
    print_2d_grid(p,10)

    print("\n\n")

FOR R = 100
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        100          |  97.20352595280221   |          10         
 97.20352595280221   |  94.75128165126046   |  88.20426099658746  
  94.4818341641876   |  92.35292956882506   |  89.76219984147792  
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


++++++++++++++++++++++++++++++++++++++++
 Terminal  |    LEFT    |  Terminal 
    UP     |    LEFT    |    DOWN   
    UP     |    LEFT    |    LEFT   
++++++++++++++++++++++++++++++++++++++++



FOR R = 3
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
         3           |   8.46193927404121   |          10         
 5.383565054208173   |  7.113204908193241   |   8.46193927404121  
 4.574815827465922   |  5.794111262853771   |  6.965008790303176  
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


++++++++++++++++++++++++++++++++++++++++
 Terminal  

Using a Discount factor 0,99 implies that the future rewards are slightly less valuable than immediate rewards

for R= 100
The reward for reaching this terminal state is much larger than any other reward, so the policy is toward the 100

for r= -3
the top left corner is a big loss , so the model will avoid it and try to reach towards the 10 reward

for r= 0
this is a neutral value, the policy will want to reach the 10 reward.

for r = 3
it is a reward but smaller than 10 reward, the policy will prefer to go to 10

## part 2 :policy iteration

Given a policy π Evaluate the policy π vπ(s) = E[Rt+1 +γRt+2 +...|St = s] Improve the policy by acting greedily with respect to vπ π=greedy(vπ)

In [8]:
def evaluate_policy(policy, grid):
   for i in range(300):
        for r in range(3):
            for c in range(3):
                if (r == 0 and c == 0) or (r == 0 and c == 2):
                    continue
                v = grid[r][c]
                V_pi = calc_v(grid, r, c, policy[r][c])
                grid[r][c] = V_pi
def improve_policy(policy, grid):
    policy_stable = True
    for r in range(3):
        for c in range(3):
            if (r == 0 and c == 0) or (r == 0 and c == 2):
                continue
            old_action = policy[r][c]
            max_value = -1000000
            best_action = 0
            for action in range(NUM_ACTIONS):
                value = calc_v(grid, r, c, action)
                if value > max_value:
                    max_value = value
                    best_action = action
            policy[r][c] = best_action


def print_policy(policy):
    for row in range(NUM_ROWS):
        for col in range(NUM_COLS):
            if (row == 0 and col == 0) or (row == 0 and col == 2):
                print("Terminal", end=" | ")
            else:
                print(ACTIONS_TEXT[policy[row][col]], end=" | ")
        print()

r=100
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print("Final Policy:")
print_policy(policy)



Final Policy:
Terminal | LEFT | Terminal | 
UP | LEFT | DOWN | 
UP | LEFT | LEFT | 


In [9]:
r=10
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print("Final Policy:")
print_policy(policy)

Final Policy:
Terminal | RIGHT | Terminal | 
UP | UP | UP | 
UP | UP | UP | 


In [10]:
r=3
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print("Final Policy:")
print_policy(policy)

Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 


In [11]:
r=0
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print("Final Policy:")
print_policy(policy)

Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 


In [12]:
r=-3
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print("Final Policy:")
print_policy(policy)

Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 


In [18]:
def evaluate_policy(policy, env_v):
    delta = float('inf')
    while delta > 0.01:
        delta = 0
        for r in range(NUM_ROWS):
            for c in range(NUM_COLS):
                if (r == 0 and c == 0) or (r == 0 and c == 2):
                    continue
                v = env_v[r][c]
                new_v = calc_v(env_v, r, c, policy[r][c])
                env_v[r][c] = new_v
                delta = max(delta, abs(v - new_v))

def improve_policy(policy, env_v):
    policy_stable = True
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS):
            if (r == 0 and c == 0) or (r == 0 and c == 2):
                continue
            old_action = policy[r][c]
            max_action, max_u = None, -float("inf")
            for action in range(NUM_ACTIONS):
                u = calc_v(env_v, r, c, action)
                if u > max_u:
                    max_action, max_u = action, u
            policy[r][c] = max_action



def print_policy(policy):
    for row in range(NUM_ROWS):
        for col in range(NUM_COLS):
            if (row == 0 and col == 0) or (row == 0 and col == 2):
                print("Terminal", end=" | ")
            else:
                print(ACTIONS_TEXT[policy[row][col]], end=" | ")
        print()

r=100
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    i=i+1
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print(i)
print("Final Policy:")
print_policy(policy)

print("\nState Values:")
print_2d_grid(env_v, 10)


100
Final Policy:
Terminal | LEFT | Terminal | 
UP | LEFT | DOWN | 
UP | LEFT | LEFT | 

State Values:
++++++++++++++++++++++++++++++++++++++++
   100     | 97.20352595280221 |     10    
97.20352595280221 | 94.75128165126046 | 88.20426099658746
94.4818341641876 | 92.35292956882506 | 89.76219984147792
++++++++++++++++++++++++++++++++++++++++


In [19]:
r=10
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    i=i+1
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print(i)
print("Final Policy:")
print_policy(policy)

print("\nState Values:")
print_2d_grid(env_v, 10)

100
Final Policy:
Terminal | RIGHT | Terminal | 
UP | UP | UP | 
UP | UP | UP | 

State Values:
++++++++++++++++++++++++++++++++++++++++
    10     | 8.494501799524278 |     10    
8.494501799524278 | 7.4095567815290355 | 8.494501799524278
7.045187251785636 | 6.263316046824553 | 7.045187251785636
++++++++++++++++++++++++++++++++++++++++


In [20]:
r=3
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    i=i+1
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print(i)
print("Final Policy:")
print_policy(policy)

print("\nState Values:")
print_2d_grid(env_v, 10)

100
Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 

State Values:
++++++++++++++++++++++++++++++++++++++++
    3      | 8.46193927404121 |     10    
5.383565054208173 | 7.113204908193241 | 8.46193927404121
4.574815827465922 | 5.794111262853771 | 6.965008790303176
++++++++++++++++++++++++++++++++++++++++


In [21]:
r=0
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    i=i+1
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print(i)
print("Final Policy:")
print_policy(policy)

print("\nState Values:")
print_2d_grid(env_v, 10)

100
Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 

State Values:
++++++++++++++++++++++++++++++++++++++++
    0      | 8.46193927404121 |     10    
5.083298783711909 | 7.113204908193241 | 8.46193927404121
4.541823196190527 | 5.794111262853771 | 6.965008790303176
++++++++++++++++++++++++++++++++++++++++


In [22]:
r=-3
env_v = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
policy = [[0, 0, 0] for _ in range(NUM_ROWS)]  # Initialize policy arbitrarily
policy_stable = False
i=0
for k in range(100):
    i=i+1
    evaluate_policy(policy, env_v)
    improve_policy(policy, env_v)
print(i)
print("Final Policy:")
print_policy(policy)

print("\nState Values:")
print_2d_grid(env_v, 10)

100
Final Policy:
Terminal | RIGHT | Terminal | 
RIGHT | RIGHT | UP | 
RIGHT | RIGHT | UP | 

State Values:
++++++++++++++++++++++++++++++++++++++++
    -3     | 8.46193927404121 |     10    
4.7830325132156455 | 7.113204908193241 | 8.46193927404121
4.508830564915133 | 5.794111262853771 | 6.965008790303176
++++++++++++++++++++++++++++++++++++++++




* For value iteration Complexity O(mn2) per iteration, for m actions and n states
*  For policy iteration Complexity O (m2n2) per iteration



## Comparsion between value iteration and policy iteration



1. Policy Iteration

*  Starts with a random policy
*  Algorithm is more complex


*   Guaranteed to converge
*   Cheaper to compute

*   Requires few iterations to converge
*   Faster
----------------------------------------------------------


2.  Value Iteration

*  Starts with a random value function

*  Algorithm is simpler

*   Guaranteed to converge
*   More expensive to compute


*   Requires more iterations to converge


*   Slower








